<a href="https://colab.research.google.com/github/invisibleyoubic/learnsymbol_Nikulin/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Math
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def check(x):
  if isinstance(x, Expression):
    return x
  return Expression(x)

class Expression:
  def __init__(self, f):
    self.f = f
  def __add__(self, x):
    x = check(x)
    return Expression(add(self.f, x.f))
  def __sub__(self, x):
    x = check(x)
    return Expression(sub(self.f, x.f))
  def __mul__(self, x):
    x = check(x)
    return Expression(mul(self.f, x.f))
  def __truediv__(self, x):
    x = check(x)
    return Expression(div(self.f, x.f))
  def __pow__(self, x):
    x = check(x)
    return Expression(pow(self.f, x.f))
  def __radd__(self, x):
    x = check(x)
    return Expression(add(x.f, self.f))
  def __rsub__(self, x):
    x = check(x)
    return Expression(sub(x.f, self.f))
  def __rmul__(self, x):
    x = check(x)
    return Expression(mul(x.f, self.f))
  def __rtruediv__(self, x):
    x = check(x)
    return Expression(div(x.f, self.f))
  def __rpow__(self, x):
    x = check(x)
    return Expression(pow(x.f, self.f))
  def __neg__(self):
    return Expression(neg(self.f))
  def subs(self, var, val):
    return Expression(substitute(self.f, var, val))
  def evalf(self):
    return evaluate(self.f)
  def __str__(self):
    return expr2latex(self.f)
  def show(self):
    display(Math(expr2latex(self.f)))
  def diff(self, var):
    var = check(var).f
    return Expression(differentiate(self.f, var))
  def simp(self):
    return Expression(simplify(self.f))
  def expn(self):
    return Expression(expand(self.f))
  

In [ ]:
def add(x, y):
  return ['+', x, y]

def mul(x, y):
  return ['*', x, y]

def div(x, y):
  return ["/", x, y]

def sub(x, y):
  return ["-", x, y]

def pow(x, y):
  return ["^", x, y]

def neg(x):
  return ["--", x]

In [ ]:
def fsqrt(x):
  return ['sqrt', x]

def fsin(x):
  return ['sin', x]

def fcos(x):
  return ['cos', x]

def fln(x):
  return ['ln', x]

In [ ]:
def sqrt(x):
  x = check(x)
  t = x.f
  r = fsqrt(t)
  return Expression(r)

def sin(x):
  return Expression(fsin(check(x).f))

def cos(x):
  return Expression(fcos(check(x).f))

def ln(x):
  return Expression(fln(check(x).f))

In [ ]:
def substitute(f, var, val):
  val = check(val).f
  if f == var:
    return val
  elif isinstance(f, list):
    y = [f[0]]
    for i in range(1, len(f)):
      t = substitute(f[i], var, val)
      y.append(t)
    return y
  else:
    return f

In [ ]:
import math

def evaluate(f):
  if isinstance(f, (int, float)):
    return f
  elif isinstance(f, list):
    if f[0] == "+": 
      return evaluate(f[1]) + evaluate(f[2])
    elif f[0] == "*":
      return evaluate(f[1]) * evaluate(f[2])
    elif f[0] == "sqrt":
      return math.sqrt(evaluate(f[1]))
    elif f[0] == "sin":
      return math.sin(evaluate(f[1]))
    elif f[0] == "cos":
      return math.cos(evaluate(f[1]))
  return None
  

In [ ]:
def symbols(vars):
  return map(Expression, vars.split())

x, y = symbols("x y")  

In [ ]:
def simplify(f):
  if isinstance(f, list):
    r = [f[0]]
    for x in f[1:]:
      r.append(simplify(x))
    if r[0] == "+":
      if isinstance(r[1], int) and isinstance(r[2], int):
        return r[1] + r[2] 
      if r[1] == 0: 
        return r[2]
      if r[2] == 0:
        return r[1]
      if r[1] == r[2]:
        return mul(2, r[1])
      if isinstance(r[2], list) and r[2][0] == "--": # unary minus
        return simplify(sub(r[1], r[2][1]))
    if r[0] == "-":
      if isinstance(r[1], int) and isinstance(r[2], int):
        return r[1] - r[2]
      if r[1] == 0: 
        return neg(r[2])
      if r[2] == 0:
        return r[1]
      if r[1] == r[2]:
        return 0
      if isinstance(r[2], list) and r[2][0] == "--": # unary minus
        return simplify(add(r[1], r[2][1]))
    if r[0] == "*":
      if isinstance(r[1], int) and isinstance(r[2], int):
        return r[1] * r[2] 
      if r[1] == 0 or r[2] == 0: 
        return 0
      if r[1] == 1:
        return r[2]
      if r[2] == 1:
        return r[1]
      if r[1] == r[2]:
        return pow(r[1], 2)
    if r[0] == "^":
      if isinstance(r[1], int) and isinstance(r[2], int) and r[2] > 0:
        return r[1] ** r[2] 
      if r[2] == 1: 
        return r[1]
    if r[0] == "sqrt":
      if isinstance(r[1], int):
        a = int(math.sqrt(r[1]))
        if a * a == r[1]:
          return a
        return r 
    return r
  return f 

In [ ]:
def expr2latex(x):
  if isinstance(x, (int, float)):
    return str(x)
  if isinstance(x, str):
    return x
  if isinstance(x, list):
    if x[0] == "+":
      return expr2latex(x[1]) + "+" + expr2latex(x[2])
    if x[0] == "-":
      return expr2latex(x[1]) + "-" + expr2latex(x[2])
    if x[0] == "*":
      left = expr2latex(x[1])
      if isinstance(x[1], list) and x[1][0] == "+":
        left = "\\left(" + left + "\\right)"
      right = expr2latex(x[2])
      if isinstance(x[2], list) and x[2][0] == "+":
        right = "\\left(" + right + "\\right)"
      return left + "\\cdot " + right
    if x[0] == "/":
      return "\\dfrac{" + expr2latex(x[1]) + "}{" + expr2latex(x[2]) + "}"
    if x[0] == "^":
      left = expr2latex(x[1])
      if isinstance(x[1], list):
        left = "\\left(" + left + "\\right)"
      right = expr2latex(x[2])
      return "{" + left + "}^{" + right + "}"
    if x[0] == "--":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "-{" + a + "}"
    if x[0] == "sqrt":
      a = expr2latex(x[1])
      return "\\sqrt{" + a + "}"
    if x[0] == "sin":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "\\sin{" + a + "}"
    if x[0] == "cos":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "\\cos{" + a + "}"
    if x[0] == "ln":
      a = expr2latex(x[1])
      if isinstance(x[1], list):
        a = "\\left(" + a + "\\right)"
      return "\\ln{" + a + "}"


In [ ]:
def expand(f):
  if isinstance(f, list):
    r = [f[0]]
    for x in f[1:]:
      r.append(expand(x))
    if r[0] == "*":
      if isinstance(r[1], list) and r[1][0] == "+":
        a = expand(mul(r[1][1], r[2]))
        b = expand(mul(r[1][2], r[2]))
        return add(a, b)
      if isinstance(r[2], list) and r[2][0] == "+":
        a = expand(mul(r[1], r[2][1]))
        b = expand(mul(r[1], r[2][2]))
        return add(a, b)
    if r[0] == "^":
      if isinstance(r[1], list) and r[1][0] == "+" and r[2] == 2:
        a = pow(r[1][1], 2)
        b = mul(2, mul(r[1][1], r[1][2]))
        c = pow(r[1][2], 2)
        return add(a, add(b, c))
    return r
  return f


In [ ]:
z = x--x
print(z.f)
z.show()
z.simp().show()

['-', 'x', ['--', 'x']]


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
z = sqrt(cos(x))
z.simp().show()
print(z.f)
z.show()
z.simp().show()

<IPython.core.display.Math object>

['sqrt', ['cos', 'x']]


<IPython.core.display.Math object>

<IPython.core.display.Math object>